In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import copy

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor
from xgboost import XGBClassifier

import statsmodels.api as sm

In [36]:
ITI_amount = pd.read_csv('./data/iti/ITI_amount.csv')
ITI_count = pd.read_csv('./data/iti/ITI_count.csv')
ITI_flag = pd.read_csv('./data/iti/ITI_flag.csv')

In [37]:
ITI_amount = ITI_amount.set_index('date')
ITI_count = ITI_count.set_index('date')
ITI_flag = ITI_flag.set_index('date')

ITI_count

,ITI_count
date,
2018-01-02,-0.000855
2018-01-03,0.009956
2018-01-04,0.017666
2018-01-05,0.011361
2018-01-06,0.021734
...,...
2024-09-26,0.036592
2024-09-27,0.042505
2024-09-28,0.020111


In [38]:
reversion_df = pd.read_csv('./data/reversion/reversion.csv')
reversion_df = reversion_df[['Open datetime', 'Reversion_Signals']]

In [39]:
reversion_df.columns = ['date', 'target']
reversion_df = reversion_df.set_index('date')
reversion_df

,target
date,
2018-01-07,1
2018-01-08,0
2018-01-09,0
2018-01-10,1
2018-01-11,1
...,...
2024-09-26,0
2024-09-27,0
2024-09-28,0


In [40]:
result_df = pd.concat([ITI_amount, ITI_count, ITI_flag, reversion_df], axis=1)
result_df = result_df.dropna()
result_df

,ITI_amount,ITI_count,ITI_flag,target
date,,,,
2018-01-07,0.003948,0.010048,0.992118,1.0
2018-01-08,0.006804,0.013995,1.003961,0.0
2018-01-09,0.003948,0.009246,0.028513,0.0
2018-01-10,0.003948,0.010008,0.011397,1.0
2018-01-11,0.003948,0.007570,0.981418,1.0
...,...,...,...,...
2024-09-26,0.015272,0.036592,0.394437,0.0
2024-09-27,0.022266,0.042505,0.732642,0.0
2024-09-28,0.017516,0.020111,0.690499,0.0


In [41]:
y = result_df['target'].values
X = result_df[['ITI_amount', 'ITI_count', 'ITI_flag']]

In [42]:
## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=0)

In [43]:
X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
# print("=== OLS 기본 요약 ===")
# print(lr.summary())

# 로버스트 공분산(이분산 robust: HC1) 적용
# ---------------------------------------------
lr_hc1 = lr.get_robustcov_results(cov_type='HC1')
print("\n=== 로버스트(HC1) 요약 ===")
print(lr_hc1.summary())



=== 로버스트(HC1) 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     3.439
Date:                Tue, 04 Feb 2025   Prob (F-statistic):             0.0168
Time:                        18:53:31   Log-Likelihood:                -61.345
No. Observations:                 492   AIC:                             130.7
Df Residuals:                     488   BIC:                             147.5
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0612      0.0

In [44]:
corr_df = result_df[['ITI_amount', 'ITI_count', 'ITI_flag']]
corr_df.corr()

,ITI_amount,ITI_count,ITI_flag
ITI_amount,1.000000,0.144328,0.167909
ITI_count,0.144328,1.000000,0.267994
ITI_flag,0.167909,0.267994,1.000000


In [45]:
test_df = pd.read_csv("./data/test/test_df.csv")
test_df = test_df.set_index('date')

In [46]:
test_df['volatility_5'] = test_df['returns'].rolling(window=5).std()
test_df['volatility_10'] = test_df['returns'].rolling(window=10).std()
test_df['volatility_15'] = test_df['returns'].rolling(window=15).std()
test_df['volatility_30'] = test_df['returns'].rolling(window=30).std()
test_df['volatility_60'] = test_df['returns'].rolling(window=60).std()

pd.set_option('display.max_columns', None)

# nan값 많은 열
# 'cost-per-transaction-percent'
# 'estimated-transaction-volume-usd'
# 'estimated-transaction-volume'

test_df = test_df.drop(['cost-per-transaction-percent', 'estimated-transaction-volume-usd', 'estimated-transaction-volume'], axis=1)

test_df

,transaction_count,transaction_amount,transaction_flag,returns,close,abs_returns,volume,open,high,low,high_low,daily_avg_trades,daily_sum_trades,daily_sum_taker_buy_base_asset_volume,daily_sum_taker_buy_quote_asset_volume,daily_avg_spread,daily_quote_asset_volume,daily_avg_quote_asset_volume,daily_low,daily_high,daily_avg_price,daily_vwap,daily_avg_volume,daily_morning_returns,daily_afternoon_returns,daily_night_returns,daily_dawn_returns,daily_avg_morning_returns,daily_avg_afternoon_returns,daily_avg_night_returns,daily_avg_dawn_returns,daily_avg_morning_volume,daily_avg_afternoon_volume,daily_avg_night_volume,daily_avg_dawn_volume,daily_morning_volatility,daily_afternoon_volatility,daily_night_volatility,daily_dawn_volatility,daily_volatility,avg-block-size,blocks-size,cost-per-transaction,difficulty,fees-usd-per-transaction,hash-rate,market-price,median-confirmation-time,miners-revenue,n-transactions-excluding-popular,n-transactions-per-block,n-transactions-total,n-transactions,n-unique-addresses,output-volume,trade-volume,transaction-fees-usd,transaction-fees,n-payments-per-block,n-payments,avg-confirmation-time,volatility_5,volatility_10,volatility_15,volatility_30,volatility_60
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,0,0.000000e+00,0,NaN,4285.08,NaN,795.150377,4261.48,4485.39,4200.74,284.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,0,0.000000e+00,0,-0.041238,4108.37,0.041238,1199.888264,4285.08,4371.52,3938.77,432.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.944878,97293.875547,7.350606,3.176884e+11,0.394819,2.526780e+06,997.36,9.816667,2.138666e+06,287576.0,1829.880503,1.839618e+08,290951.0,529709.0,1.573982e+06,6.822471e+06,114872.988609,112.884147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,0,0.000000e+00,0,0.007694,4139.98,0.007694,381.309763,4108.37,4184.69,3850.00,334.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.906574,97444.110856,7.415466,3.176884e+11,0.398843,2.589950e+06,1013.42,12.350000,2.245225e+06,297536.0,1839.414634,1.842534e+08,301664.0,515024.0,1.950525e+06,8.324526e+06,120759.971352,117.508697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,0,0.000000e+00,0,-0.012969,4086.29,0.012969,467.083022,4139.98,4211.08,4032.62,178.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.986063,97592.788943,7.000531,3.176884e+11,0.644994,2.432026e+06,1037.83,10.700000,2.332472e+06,325192.0,2147.986928,1.845561e+08,328642.0,545609.0,2.482038e+06,9.340584e+06,214902.266038,193.337436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,1,3.936639e+13,1,-0.017201,4016.00,0.017201,691.743060,4086.29,4119.62,3911.79,207.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.984197,97743.656720,6.913398,3.176884e+11,0.419516,2.210933e+06,1109.16,10.916667,2.009801e+06,283920.0,2060.721429,1.848845e+08,288501.0,495910.0,2.998215e+06,2.387011e+07,121958.044759,114.991943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-26,0,0.000000e+00,0,0.032018,65173.99,0.032018,28373.305930,63152.01,65839.00,62670.00,3169.00,50.478391,4361333.0,15041.98860,9.711762e+08,0.000005,1.831205e+09,21194.497638,62670.00,65839.00,64278.054721,64540.884675,0.328395,0.012580,0.011469,0.000636,0.006999,5.796342e-07,5.319940e-07,3.103639e-08,3.237245e-07,0.251817,0.666416,0.210727,0.184619,0.000042,0.000

In [47]:
ITI_amount = pd.read_csv("./data/iti/ITI_amount_241226.csv")
ITI_count = pd.read_csv("./data/iti/ITI_count_241226.csv")
ITI_flag = pd.read_csv("./data/iti/ITI_flag_241226.csv")

ITI_amount = ITI_amount.set_index('date')
ITI_count = ITI_count.set_index('date')
ITI_flag = ITI_flag.set_index('date')


In [48]:
# test_target_df = test_df[['volatility_5', 'volatility_10', 'volatility_15', 'volatility_30', 'volatility_60']]
test_target_df = pd.concat([ITI_amount, ITI_count, ITI_flag, test_df], axis=1)
test_target_df = test_target_df.dropna()
test_target_df['volatility_5'] = test_target_df['volatility_5'].shift(-5)
test_target_df['volatility_10'] = test_target_df['volatility_10'].shift(-10)
test_target_df['volatility_15'] = test_target_df['volatility_15'].shift(-15)
test_target_df['volatility_30'] = test_target_df['volatility_30'].shift(-30)
test_target_df['volatility_60'] = test_target_df['volatility_60'].shift(-60)

# test_target_df[['transaction_count','transaction_amount','transaction_flag']] = test_target_df[['transaction_count','transaction_amount','transaction_flag']].shift(-1)
test_target_df = test_target_df.dropna()
test_target_df

,ITI_amount,ITI_count,ITI_flag,transaction_count,transaction_amount,transaction_flag,returns,close,abs_returns,volume,open,high,low,high_low,daily_avg_trades,daily_sum_trades,daily_sum_taker_buy_base_asset_volume,daily_sum_taker_buy_quote_asset_volume,daily_avg_spread,daily_quote_asset_volume,daily_avg_quote_asset_volume,daily_low,daily_high,daily_avg_price,daily_vwap,daily_avg_volume,daily_morning_returns,daily_afternoon_returns,daily_night_returns,daily_dawn_returns,daily_avg_morning_returns,daily_avg_afternoon_returns,daily_avg_night_returns,daily_avg_dawn_returns,daily_avg_morning_volume,daily_avg_afternoon_volume,daily_avg_night_volume,daily_avg_dawn_volume,daily_morning_volatility,daily_afternoon_volatility,daily_night_volatility,daily_dawn_volatility,daily_volatility,avg-block-size,blocks-size,cost-per-transaction,difficulty,fees-usd-per-transaction,hash-rate,market-price,median-confirmation-time,miners-revenue,n-transactions-excluding-popular,n-transactions-per-block,n-transactions-total,n-transactions,n-unique-addresses,output-volume,trade-volume,transaction-fees-usd,transaction-fees,n-payments-per-block,n-payments,avg-confirmation-time,volatility_5,volatility_10,volatility_15,volatility_30,volatility_60
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-02,0.003682,0.005105,0.939970,4,3.744000e+14,1,0.097050,32178.33,0.097050,129993.873362,29331.70,33300.00,28946.53,4353.47,25.994468,2245922.0,67446.305246,2.110335e+09,0.000065,4.073842e+09,47150.950968,28946.53,33300.00,30694.541181,31339.198481,1.504559,0.005577,0.104250,-0.020654,0.008806,2.629675e-07,4.620890e-06,-9.149101e-07,4.097032e-07,0.709484,2.586270,2.205785,0.516695,0.000105,0.000244,0.000320,0.000087,0.000213,1.395637,319315.580633,107.041738,1.859959e+13,8.400989,1.396132e+08,29393.75,11.625000,3.190850e+07,293409.0,1967.622517,601835959.0,297111.0,711719.0,2.164453e+06,4.912580e+08,2.504284e+06,79.699067,4414.298013,666559.0,87.547595,0.046184,0.054750,0.054983,0.053360,0.054240
2021-01-03,0.000717,0.003121,0.982070,1,2.040000e+12,1,0.025536,33000.05,0.025536,120957.566750,32176.45,34778.11,31962.99,2815.12,27.427060,2369698.0,59750.332871,2.004428e+09,0.000079,4.057598e+09,46962.944739,31962.99,34778.11,33449.583730,33545.837360,1.399972,0.019398,-0.043711,0.002758,0.049112,9.142759e-07,-2.040671e-06,1.486381e-07,2.236047e-06,1.627154,1.790353,1.039721,1.142659,0.000223,0.000239,0.000205,0.000181,0.000213,1.288694,319526.405445,100.029960,1.859959e+13,8.057991,1.460853e+08,32195.46,15.083333,3.592236e+07,355123.0,2272.886076,602133929.0,359116.0,812749.0,2.211805e+06,1.393907e+09,2.893754e+06,86.419994,4742.778481,749359.0,111.085992,0.045881,0.061740,0.054920,0.054156,0.054547
2021-01-04,0.003189,0.006457,0.020893,0,0.000000e+00,0,-0.030647,31988.71,0.030647,140899.885690,33000.05,33600.00,28130.00,5470.00,30.583426,2642408.0,69088.469230,2.173435e+09,0.000121,4.429010e+09,51261.693862,28130.00,33600.00,31826.481466,31431.117838,1.630786,-0.063083,0.009144,0.031090,-0.005668,-2.759788e-06,4.608122e-07,1.437108e-06,-2.483983e-07,2.816325,1.839324,0.863119,1.004376,0.000717,0.000281,0.000198,0.000172,0.000407,1.318806,319729.832857,103.580664,1.859959e+13,9.162334,1.599542e+08,33000.78,12.883333,3.975281e+07,368932.0,2160.312139,602493027.0,373734.0,851667.0,1.922522e+06,9.758314e+08,3.516375e+06,108.705036,4622.219653,799644.0,85.474696,0.038638,0.060359,0.054454,0.054716,0.054313
2021-01-05,0.007217,0.001479,0.003064,0,0.000000e+00,0,0.061297,33949.53,0.061297,116049.997038,31989.75,34360.00,29900.00,4460.00,29.245961,2526851.0,59691.754755,1.927195e+09,0.000073,3.743617e+09,43328.902137,29900.00,34360.00,32221.813669,32258.733179,1.343171,0.044040,0.030402,0.036857,-0.048531,2.021824e-06,1.406496e-06,1.702368e-06,-2.263027e-06,1.049916,1.146717,1.583017,1.593034,0.000230,0.000200,0.000231,0.000283,0.000238,1.292713,319957.982033,99.733618,1.859959e+13,10.386146,1.451608e+08,32035.03,13.650000,3.531478e+07,349895.0,225

In [49]:
test_target_df.columns

Index(['ITI_amount', 'ITI_count', 'ITI_flag', 'transaction_count',
       'transaction_amount', 'transaction_flag', 'returns', 'close',
       'abs_returns', 'volume', 'open', 'high', 'low', 'high_low',
       'daily_avg_trades', 'daily_sum_trades',
       'daily_sum_taker_buy_base_asset_volume',
       'daily_sum_taker_buy_quote_asset_volume', 'daily_avg_spread',
       'daily_quote_asset_volume', 'daily_avg_quote_asset_volume', 'daily_low',
       'daily_high', 'daily_avg_price', 'daily_vwap', 'daily_avg_volume',
       'daily_morning_returns', 'daily_afternoon_returns',
       'daily_night_returns', 'daily_dawn_returns',
       'daily_avg_morning_returns', 'daily_avg_afternoon_returns',
       'daily_avg_night_returns', 'daily_avg_dawn_returns',
       'daily_avg_morning_volume', 'daily_avg_afternoon_volume',
       'daily_avg_night_volume', 'daily_avg_dawn_volume',
       'daily_morning_volatility', 'daily_afternoon_volatility',
       'daily_night_volatility', 'daily_dawn_volatili

In [50]:
y = test_target_df['volatility_5'].values
X = test_target_df[['ITI_amount', 'ITI_count', 'ITI_flag']]

## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS 기본 요약 ===")
print(lr.summary())

=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.4843
Date:                Tue, 04 Feb 2025   Prob (F-statistic):              0.694
Time:                        18:53:31   Log-Likelihood:                 670.27
No. Observations:                 207   AIC:                            -1333.
Df Residuals:                     203   BIC:                            -1319.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0194      0.002  

In [51]:
import time
from IPython.display import clear_output
for t in test_target_df.columns:
    # 출력 지우기 (덮어쓰기 효과)
    clear_output(wait=True)
    y = test_target_df['volatility_5'].values
    X = test_target_df[['ITI_amount', 'ITI_count', 'ITI_flag', t]]

    ## train, test 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

    X_with_const = sm.add_constant(X_test)

    # OLS 회귀 적합
    # ---------------------------------------------
    lr = sm.OLS(y_test, X_with_const).fit()
    print("=== OLS 기본 요약 ===")
    print(lr.summary())

    # # 로버스트 공분산(이분산 robust: HC1) 적용
    # # ---------------------------------------------
    # lr_hc1 = lr.get_robustcov_results(cov_type='HC1')
    # print("\n=== 로버스트(HC1) 요약 ===")
    # print(lr_hc1.summary())


    time.sleep(3)


=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     15.17
Date:                Tue, 04 Feb 2025   Prob (F-statistic):           2.51e-11
Time:                        18:53:58   Log-Likelihood:                 978.94
No. Observations:                 310   AIC:                            -1948.
Df Residuals:                     305   BIC:                            -1929.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0172      0.002  

KeyboardInterrupt: 

In [396]:
reversion_df = pd.read_csv('./data/reversion/reversion_241226.csv')
reversion_df = reversion_df[['Open datetime', 'Reversion_Signals']]

reversion_df.columns = ['date', 'target']
reversion_df = reversion_df.set_index('date')

result_df = pd.concat([ITI_amount, ITI_count, ITI_flag, reversion_df], axis=1)
result_df = result_df.dropna()

y = result_df['target'].values
X = result_df[['ITI_amount', 'ITI_count', 'ITI_flag']]

## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
# print("=== OLS 기본 요약 ===")
# print(lr.summary())

# 로버스트 공분산(이분산 robust: HC1) 적용
# ---------------------------------------------
lr_hc1 = lr.get_robustcov_results(cov_type='HC1')
print("\n=== 로버스트(HC1) 요약 ===")
print(lr_hc1.summary())



=== 로버스트(HC1) 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     5.719
Date:                Fri, 27 Dec 2024   Prob (F-statistic):           0.000716
Time:                        09:35:12   Log-Likelihood:                -142.79
No. Observations:                 738   AIC:                             293.6
Df Residuals:                     734   BIC:                             312.0
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0577      0.0

In [400]:
y = test_target_df['transaction_count'].shift(-1).fillna(test_target_df['transaction_count'].mean()).values
# y = test_target_df['transaction_amount']
X = test_target_df[['ITI_amount', 'ITI_count', 'ITI_flag']]

## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS 기본 요약 ===")
print(lr.summary())

=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.017
Date:                Fri, 27 Dec 2024   Prob (F-statistic):              0.385
Time:                        09:36:13   Log-Likelihood:                -416.61
No. Observations:                 310   AIC:                             841.2
Df Residuals:                     306   BIC:                             856.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1217      0.148  

In [404]:
y = test_target_df['volatility_5'].values
X = test_target_df[['transaction_amount', 'transaction_count', 'transaction_flag']]

## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS 기본 요약 ===")
print(lr.summary())

=== OLS 기본 요약 ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.6375
Date:                Fri, 27 Dec 2024   Prob (F-statistic):              0.591
Time:                        10:47:24   Log-Likelihood:                 951.77
No. Observations:                 310   AIC:                            -1896.
Df Residuals:                     306   BIC:                            -1881.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const             